# Derive models of drone

Do all imports.

In [1]:
import sympy as sym
import numpy as np
from scipy import linalg
from scipy import signal

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

## Dynamic model

Define physical parameters.

In [2]:
params = {
    'm': 0.5,
    'Jx': 0.0023,
    'Jy': 0.0023,
    'Jz': 0.0040,
    'l': 0.175,
    'g': 9.81,
}

Derive the equations of motion:

In [3]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
w_in_body = sym.Matrix([w_x, w_y, w_z])

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters
m = sym.nsimplify(params['m'])
Jx = sym.nsimplify(params['Jx'])
Jy = sym.nsimplify(params['Jy'])
Jz = sym.nsimplify(params['Jz'])
l = sym.nsimplify(params['l'])
g = sym.nsimplify(params['g'])
J = sym.diag(Jx, Jy, Jz)

# rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# applied forces
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])

# applied torques
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

# equations of motion
f = sym.Matrix.vstack(
    R_body_in_world @ v_in_body,
    M @ w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() @ (tau_in_body - w_in_body.cross(J @ w_in_body)),
)

f = sym.simplify(f, full=True)

The equations of motion have this form:

$$\begin{bmatrix} \dot{p}_x \\ \dot{p}_y \\ \dot{p}_z \\ \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{v}_x \\ \dot{v}_y \\ \dot{v}_z \\ \dot{w}_x \\ \dot{w}_y \\ \dot{w}_z \end{bmatrix} = f\left(p_x, p_y, p_z, \psi, \theta, \phi, v_x, v_y, v_z, w_x, w_y, w_z, \tau_x, \tau_y, \tau_z, f_z \right)$$

Here is the function $f$:

In [4]:
f

Matrix([
[v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) - v_z*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi))],
[                                                                      -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                                 (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                              w_y*cos(phi) - w_z*sin(phi)],
[                                                                                  w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                   v_y*w_z - v_z*w_y + 981*sin(theta)/100],
[   

## Sensor model

Define the sensor model.

In [5]:
# Position of drone in world frame
p_in_world = sym.Matrix([p_x, p_y, p_z])

# Position of markers in body frame
a_in_body = sym.Matrix([l, 0, 0])  # <-- marker on front rotor
b_in_body = sym.Matrix([-l, 0, 0]) # <-- marker on rear rotor

# Position of markers in world frame
a_in_world = p_in_world + R_body_in_world @ a_in_body
b_in_world = p_in_world + R_body_in_world @ b_in_body

# Sensor model
g = sym.simplify(sym.Matrix.vstack(a_in_world, b_in_world))

The sensor model has this form:

$$o = g(p_x, p_y, p_z, \psi, \theta)$$

Here is the function $g$:

In [6]:
g

Matrix([
[p_x + 7*cos(psi)*cos(theta)/40],
[p_y + 7*sin(psi)*cos(theta)/40],
[         p_z - 7*sin(theta)/40],
[p_x - 7*cos(psi)*cos(theta)/40],
[p_y - 7*sin(psi)*cos(theta)/40],
[         p_z + 7*sin(theta)/40]])

In [7]:
f_eqcheck = f.subs(psi, 0).subs(theta, 0).subs(phi, 0)
f_eqcheck

Matrix([
[                                v_x],
[                                v_y],
[                                v_z],
[                                w_z],
[                                w_y],
[                                w_x],
[                  v_y*w_z - v_z*w_y],
[                 -v_x*w_z + v_z*w_x],
[2*f_z + v_x*w_y - v_y*w_x - 981/100],
[     10000*tau_x/23 - 17*w_y*w_z/23],
[     10000*tau_y/23 + 17*w_x*w_z/23],
[                          250*tau_z]])

In [8]:
f_eqcheck = f_eqcheck.subs(v_x, 0).subs(v_y, 0).subs(v_z, 0).subs(w_x, 0).subs(w_y, 0).subs(w_z, 0)
f_eqcheck

Matrix([
[              0],
[              0],
[              0],
[              0],
[              0],
[              0],
[              0],
[              0],
[2*f_z - 981/100],
[ 10000*tau_x/23],
[ 10000*tau_y/23],
[      250*tau_z]])

In [9]:
g_eqcheck = g.subs(psi, 0).subs(theta, 0).subs(phi, 0)
g_eqcheck

Matrix([
[p_x + 7/40],
[       p_y],
[       p_z],
[p_x - 7/40],
[       p_y],
[       p_z]])

#### Equilibrium Points

In [27]:
p_xe = 0
p_ye = 0
p_ze = 0

psi_e = 0
theta_e = 0
phi_e = 0
v_xe = 0
v_ye = 0
v_ze = 0
w_xe = 0
w_ye = 0
w_ze = 0
tau_xe = 0
tau_ye = 0
tau_ze = 0
f_ze = (981/100)/2

A_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), f.jacobian([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z]))
B_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), f.jacobian([tau_x, tau_y, tau_z, f_z]))
C_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), g.jacobian([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z]))

A = A_num(p_xe, p_ye, p_ze, psi_e, theta_e, phi_e, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze).astype(float)
B = B_num(p_xe, p_ye, p_ze, psi_e, theta_e, phi_e, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze).astype(float)
C = C_num(p_xe, p_ye, p_ze, psi_e, theta_e, phi_e, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze).astype(float)

#### Symbolic Representation of A, B, C

In [28]:
A_sym = sym.Matrix(A)
B_sym = sym.Matrix(B)
C_sym = sym.Matrix(C)

A_sym

Matrix([
[0, 0, 0, 0,    0,     0, 1.0,   0,   0,   0,   0,   0],
[0, 0, 0, 0,    0,     0,   0, 1.0,   0,   0,   0,   0],
[0, 0, 0, 0,    0,     0,   0,   0, 1.0,   0,   0,   0],
[0, 0, 0, 0,    0,     0,   0,   0,   0,   0,   0, 1.0],
[0, 0, 0, 0,    0,     0,   0,   0,   0,   0, 1.0,   0],
[0, 0, 0, 0,    0,     0,   0,   0,   0, 1.0,   0,   0],
[0, 0, 0, 0, 9.81,     0,   0,   0,   0,   0,   0,   0],
[0, 0, 0, 0,    0, -9.81,   0,   0,   0,   0,   0,   0],
[0, 0, 0, 0,    0,     0,   0,   0,   0,   0,   0,   0],
[0, 0, 0, 0,    0,     0,   0,   0,   0,   0,   0,   0],
[0, 0, 0, 0,    0,     0,   0,   0,   0,   0,   0,   0],
[0, 0, 0, 0,    0,     0,   0,   0,   0,   0,   0,   0]])

In [29]:
B_sym

Matrix([
[               0,                0,     0,   0],
[               0,                0,     0,   0],
[               0,                0,     0,   0],
[               0,                0,     0,   0],
[               0,                0,     0,   0],
[               0,                0,     0,   0],
[               0,                0,     0,   0],
[               0,                0,     0,   0],
[               0,                0,     0, 2.0],
[434.782608695652,                0,     0,   0],
[               0, 434.782608695652,     0,   0],
[               0,                0, 250.0,   0]])

In [30]:
C_sym

Matrix([
[1.0,   0,   0,      0,      0, 0, 0, 0, 0, 0, 0, 0],
[  0, 1.0,   0,  0.175,      0, 0, 0, 0, 0, 0, 0, 0],
[  0,   0, 1.0,      0, -0.175, 0, 0, 0, 0, 0, 0, 0],
[1.0,   0,   0,      0,      0, 0, 0, 0, 0, 0, 0, 0],
[  0, 1.0,   0, -0.175,      0, 0, 0, 0, 0, 0, 0, 0],
[  0,   0, 1.0,      0,  0.175, 0, 0, 0, 0, 0, 0, 0]])

#### Check if Controllable & Observable

controller

In [31]:
Wcnt = B
n = A.shape[0]
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    Wcnt = np.block([Wcnt, col])

np.linalg.matrix_rank(Wcnt) == A.shape[0]

True

observer

In [32]:
Wobs = C.T
n = A.T.shape[0]
for i in range(1, n):
    col = np.linalg.matrix_power(A.T, i) @ C.T
    Wobs = np.block([Wobs, col])

np.linalg.matrix_rank(Wobs.T) == A.T.shape[0]

True

#### K and L from LQR

In [33]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @ B.T @ P
    return K

In [34]:
A.shape, B.shape, C.shape

((12, 12), (12, 4), (6, 12))

$Q_c$ and $R_c$ for Controller; $Q_c$ = 12x12 and $R_c$ = 4x4

In [35]:
# just identity matrix
Qc_eye = np.eye(12)
Rc_eye = np.eye(4)

# if we wanna specify values
Qc1 = 0
Qc2 = 0
Qc3 = 0
Qc4 = 0
Qc5 = 0
Qc6 = 0
Qc7 = 0
Qc8 = 0
Qc9 = 0
Qc10 = 0
Qc11 = 0
Qc12 = 0

Qc_spec = np.diag([Qc1, Qc2, Qc3, Qc4, Qc5, Qc6, Qc7, Qc8, Qc9, Qc10, Qc11, Qc12])

Rc1 = 0
Rc2 = 0
Rc3 = 0
Rc4 = 0

Rc_spec = np.diag([Rc1, Rc2, Rc3, Rc4])

$K$; Controller

In [36]:
Kc = lqr(A, B, Qc_eye, Rc_eye)
Kc

array([[-0.        , -1.        , -0.        , -0.        , -0.        ,
         5.46213218, -0.        , -1.45381723, -0.        ,  1.01248497,
         0.        ,  0.        ],
       [ 1.        , -0.        ,  0.        ,  0.        ,  5.46213218,
         0.        ,  1.45381723, -0.        , -0.        ,  0.        ,
         1.01248497, -0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        , -0.        ,
        -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.        ,  1.00399203],
       [-0.        ,  0.        ,  1.        , -0.        , -0.        ,
        -0.        , -0.        ,  0.        ,  1.41421356, -0.        ,
        -0.        ,  0.        ]])

$Q_o$ and $R_o$ for Observer; $Q_o$ = 12x12, $R_o$ and 6x6

In [37]:
# just identity matrix
Qo_eye = np.eye(12)
Ro_eye = np.eye(6)

# if we wanna specify values
Qo1 = 0
Qo2 = 0
Qo3 = 0
Qo4 = 0
Qo5 = 0
Qo6 = 0
Qo7 = 0
Qo8 = 0
Qo9 = 0
Qo10 = 0
Qo11 = 0
Qo12 = 0

Qo_spec = np.diag([Qo1, Qo2, Qo3, Qo4, Qo5, Qo6, Qo7, Qo8, Qo9, Qo10, Qo11, Qo12])

Ro1 = 0
Ro2 = 0
Ro3 = 0
Ro4 = 0
Ro5 = 0
Ro6 = 0

Ro_spec = np.diag([Ro1, Ro2, Ro3, Ro4, Ro5, Ro6])

$L$; observer

In [38]:
Lo = lqr(A.T, C.T, np.linalg.inv(Qo_eye), np.linalg.inv(Ro_eye)).T
Lo

array([[ 2.92711379, -0.        , -0.21976816,  2.92711379, -0.        ,
         0.21976816],
       [-0.        ,  2.97537602,  0.        , -0.        ,  2.97537602,
         0.        ],
       [ 0.        ,  0.        ,  1.09868411,  0.        ,  0.        ,
         1.09868411],
       [ 0.        ,  2.13087076, -0.        ,  0.        , -2.13087076,
        -0.        ],
       [ 1.25581808,  0.        , -0.30429738,  1.25581808, -0.        ,
         0.30429738],
       [-0.        , -1.30543174, -0.        , -0.        , -1.30543174,
        -0.        ],
       [ 8.1162932 ,  0.        , -1.19153357,  8.1162932 , -0.        ,
         1.19153357],
       [ 0.        ,  8.35286245,  0.        ,  0.        ,  8.35286245,
         0.        ],
       [ 0.        ,  0.        ,  0.70710678,  0.        ,  0.        ,
         0.70710678],
       [-0.        , -0.70710678, -0.        , -0.        , -0.70710678,
        -0.        ],
       [ 0.67683134,  0.        , -0.20469329,  0.

In [39]:
Kc.shape, Lo.shape

((4, 12), (12, 6))

#### Check Eigvals of $F_{cnt}$ and $F_{obs}$

In [40]:
F_cnt = A-B@Kc
F_obs = A-Lo@C

all(i < 0 for i in np.linalg.eigvals(F_cnt)), all(i < 0 for i in np.linalg.eigvals(F_obs))

(True, True)

#### Check Full System Stability

In [41]:
F_sys = np.block([[A, -B@Kc],[Lo@C, A-B@Kc-Lo@C]])

all(i < 0 for i in np.linalg.eigvals(F_sys))

True

#### Checking xhat Size

In [25]:
xhat = np.array([[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]])
xhat.shape

(12, 1)

In [26]:
A@xhat + B@(-Kc@xhat) - Lo@(C@xhat)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])